In [ ]:
# For performing classification with only the feature selected by the feature selection methods and the most important features extracted the denoising autoencoder
# Create a copy of the dataset to include only the selected features by the feature selection methods and the most important features extracted from the latent space of the denoising autoencoder
df_sel_feat = df_actual.copy()


In [ ]:
# Classification using the subset of selected features dataset
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE


# Demographics features
demographics_features = ['MOTHDEM', 'FATHDEM', 'PTGENDER']

# Imaging data features
imaging_features = ['lh.Amygdala', 'wm.lh.entorhinal', 'wm.lh.inferiortemporal', 'ctx.rh.middletemporal', 'CC_Central', 'CC_Mid_Anterior', 'CC_Mid_Posterior', 'wm.rh.corpuscallosum', 'wm.rh.precentral', 'wm.rh.medialorbitofrontal', 'ctx.rh.posteriorcingulate', 'wm.rh.corpuscallosum', 'Brain.Stem', 'rh.VentralDC',  'ctx.rh.supramarginal', 'wm.rh.transversetemporal', 'rh.Cerebellum.Cortex', 'wm.lh.precentral', 'wm.rh.lingual', 'wm.lh.inferiortemporal', 'rh.Amygdala', 'lh.Thalamus.Proper', 'ctx.rh.lateralorbitofrontal']

# Cognitive features
cognitive_features = ['adas_Q4SCORE', 'PHC_EXF', 'adas_Q10SCORE']

# genetic features
genetic_features = ['APOE']

# Dementia rating
dementia_rating = ['CDR']
# Combine the selected features into one list
selected_features = demographics_features + imaging_features + cognitive_features + genetic_features + dementia_rating

# Combining selected features with the target variable 'AD_LABEL'
df_selected = df_sel_feat[selected_features + ['AD_LABEL']]


# Separating features and target variable
X = df_selected.drop(columns=['AD_LABEL'])
y = df_selected['AD_LABEL']

# Balancing the classes using SMOTE
smote = SMOTE(random_state=40)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initializing the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=40)

# Performing k-fold cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=40)
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=kfold)

# Training the classifier
rf_classifier.fit(X_train, y_train)

# Evaluating the classifier on the test set
y_pred = rf_classifier.predict(X_test)

# Calculating accuracy, precision, recall, F1 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Printing the cross-validation scores, mean, and standard deviation
print("Cross-validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Std CV Accuracy:", cv_scores.std())

# Printing accuracy, precision, recall, F1 score
print("Test Set Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Printing confusion matrix and classification report
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)



Cross-validation Scores: [0.88636364 0.94252874 0.94252874 0.93103448 0.94252874 0.93103448
 0.89655172 0.86206897 0.90804598 0.93103448]
Mean CV Accuracy: 0.9173719958202715
Std CV Accuracy: 0.026455580089184128
Test Set Accuracy: 0.8899082568807339
Precision: 0.8944384295401447
Recall: 0.8899082568807339
F1 Score: 0.8883415666901904
Confusion Matrix:
 [[59  4  2]
 [ 6 64 11]
 [ 0  1 71]]
Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.91      0.91        65
           2       0.93      0.79      0.85        81
           3       0.85      0.99      0.91        72

    accuracy                           0.89       218
   macro avg       0.89      0.89      0.89       218
weighted avg       0.89      0.89      0.89       218



In [ ]:
# The accuracy obtained for the feature selected dataset is 89%. This indicates that the model correctly classified about 89% of the instances in the test set.
# Now, focusing on class 3 (AD):

# Precision (0.85): Among instances predicted as AD, 85% were correctly classified as AD, while the remaining 15% were incorrectly classified as non-AD.
# Recall (0.99): Out of all actual instances of AD, the model correctly identified 99% of them as AD, while only 1% were incorrectly classified as non-AD.
# F1-score (0.91): The F1-score, which is the harmonic mean of precision and recall, is 91% for class 3. It indicates a good balance between precision and recall for AD classification.
# These metrics suggest that the model performs exceptionally well in identifying cases of AD (class 3), with high precision, recall, and F1-score.

In [ ]:
# The original dataset exhibited remarkable recall (1.00) and high precision (0.73) for the AD class, indicating its ability to effectively identify and accurately predict AD cases.
# Similarly, the feature-selected dataset demonstrated high recall (0.99), suggesting that feature selection techniques retained crucial information for AD prediction. Among datasets with missingness introduced into specific columns ('MOTHDEM', 'APOE', and 'CDR'), those with 40% missingness generally showed higher recall for AD compared to datasets with higher missingness levels. Although precision remained relatively stable across missingness levels, effective management of missing data is crucial for maintaining classification performance, particularly for early AD diagnosis and intervention.
# Notably, the 'MOTHDEM' column with 40% missingness achieved the highest recall for AD (0.71), underscoring its potential utility in AD classification tasks.